#### Загрузите данные ex9_movies.mat из файла.

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
DATA_PATH = '/home/ataleckij/Projects/university/mo/Data/Lab 9/'

In [4]:
files_names = os.listdir(DATA_PATH)
print(files_names)
files_paths = [os.path.join(DATA_PATH, name) for name in files_names]

['ex9_movies.mat', 'movie_ids.txt']


In [5]:
from scipy.io import loadmat

data0 = loadmat(files_paths[0])
Y, R, = data0['Y'], data0['R']
Y.shape, R.shape

((1682, 943), (1682, 943))

#### Выберите число признаков фильмов (n) для реализации алгоритма коллаборативной фильтрации.

In [6]:
n = 10

In [7]:
nU, nM = Y.shape

ETA = np.random.randn(nU, n)
X = np.random.randn(nM, n)

In [8]:
Y

array([[5, 4, 0, ..., 5, 0, 0],
       [3, 0, 0, ..., 0, 0, 5],
       [4, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

#### Реализуйте функцию стоимости для алгоритма.
#### Реализуйте функцию вычисления градиентов.
#### При реализации используйте векторизацию для ускорения процесса обучения.
#### Добавьте L2-регуляризацию в модель.

In [9]:
def loss(ETA, X, Y, R, lambda_):
    return 1/2 * ((ETA.dot(X.T) * R - Y) ** 2).sum() \
            + lambda_/2 * (ETA ** 2).sum() \
            + lambda_/2 * (X ** 2).sum()

In [10]:
def grad_tape_ETA(ETA, X, Y, R, lr, lambda_):
    new_ETA = ETA.copy()
    new_ETA -= lr * ((ETA.dot(X.T) * R - Y).dot(X) + lambda_ * ETA)
    return new_ETA, X

def grad_tape_X(ETA, X, Y, R, lr, lambda_):
    new_X = X.copy()
    new_X -= lr * ((ETA.dot(X.T) * R - Y).T.dot(ETA) + lambda_ * X)
    return ETA, new_X

In [11]:
ETA_, X_ = ETA, X
loss(ETA, X, Y, R, lambda_=0.05)

1173077.1789030507

In [12]:
ETA_, X_ = grad_tape_ETA(ETA_, X_, Y, R, lr=0.001, lambda_=0.05)
ETA_, X_ = grad_tape_X(ETA_, X_, Y, R, lr=0.001, lambda_=0.05)
loss(ETA_, X_, Y, R, lambda_=0.05)

907419.5813547244

#### Обучите модель с помощью градиентного спуска или других методов оптимизации.

In [31]:
from tqdm import tqdm

def fit(ETA, X, Y, R, eps, lambda_, lr, maxiter):
    ETA, X = ETA.copy(), X.copy()
    loss_value = loss(ETA, X, Y, R, lambda_=lambda_)
#     for _ in tqdm(range(maxiter)):
    for i in range(maxiter):
        ETA, X = grad_tape_ETA(ETA, X, Y, R, lr=lr, lambda_=lambda_)
        ETA, X = grad_tape_X(ETA, X, Y, R, lr=lr, lambda_=lambda_)
        new_loss_value = loss(ETA, X, Y, R, lambda_=lambda_)
        print(i, abs(loss_value - new_loss_value))
        if abs(loss_value - new_loss_value) <= eps:
            break
        loss_value = new_loss_value
    return ETA, X

In [15]:
ETA_, X_ = fit(ETA, X, Y, R, eps=1e-5, lambda_=0.005, 
               lr=0.0005, maxiter=1000)

0 161627.7378345565
1 94481.17675789131
2 63771.09994366742
3 49212.354885967565
4 43795.72267938871
5 44825.00412420207
6 50809.83081165969
7 59618.35492894007
8 67383.44331158279
9 69536.0877082586
10 64309.18112347147
11 54483.1960007163
12 44205.29189905152
13 35613.04650471336
14 28938.384471897152
15 23795.17959239034
16 19791.67715701938
17 16636.28349372436
18 14119.061423057865
19 12087.2820528562
20 10428.76206609371
21 9060.546993243857
22 7920.860213597276
23 6963.272178580039
24 6152.487893590573
25 5461.32897518063
26 4868.585781527465
27 4357.491302003749
28 3914.6323658678593
29 3529.1655493763246
30 3192.2449082762905
31 2896.5976632771926
32 2636.2043269795686
33 2406.0536726541904
34 2201.952289661407
35 2020.374695903025
36 1858.3441252214252
37 1713.3368909239216
38 1583.2051198811241
39 1466.113963681768
40 1360.4903225021262
41 1264.9807905927737
42 1178.4170309193796
43 1099.7871631205635
44 1028.212038170379
45 962.9254981390477
46 903.2578963100314
47 848.6222

371 15.58300829930522
372 15.540570449578809
373 15.498319551887107
374 15.456254028700641
375 15.414372326275043
376 15.372672913879796
377 15.331154283725482
378 15.28981494990876
379 15.248653448084951
380 15.207668335104245
381 15.166858187840262
382 15.12622160356841
383 15.085757198641659
384 15.045463608890714
385 15.005339488205209
386 14.96538350895571
387 14.92559436114243
388 14.8859707518277
389 14.846511405303318
390 14.807215062290197
391 14.768080479516357
392 14.729106429709645
393 14.690291701037495
394 14.651635096699465
395 14.613135434883588
396 14.574791548177018
397 14.536602283580578
398 14.498566501701134
399 14.46068307722453
400 14.42295089790423
401 14.385368865041528
402 14.347935892226815
403 14.310650906591036
404 14.273512846884842
405 14.236520664540876
406 14.199673322880699
407 14.16296979708568
408 14.126409073811374
409 14.08999015136942
410 14.053712039043603
411 14.017573757235368
412 13.981574337434722
413 13.945712821492634
414 13.909988262021216

738 6.78654670137621
739 6.773770699335728
740 6.761033052891435
741 6.748333624789666
742 6.735672277944104
743 6.723048875530367
744 6.710463281109696
745 6.697915358654427
746 6.685404972286051
747 6.672931986402546
748 6.660496265980328
749 6.648097676144971
750 6.635736082207586
751 6.623411349999515
752 6.611123345723172
753 6.598871935773786
754 6.586656986826711
755 6.5744783660557005
756 6.562335940881894
757 6.5502295790211065
758 6.5381591486002435
759 6.526124518051802
760 6.51412555612842
761 6.502162131979276
762 6.490234115004569
763 6.478341375037417
764 6.466483782209252
765 6.454661207037134
766 6.442873520300054
767 6.431120593246305
768 6.419402297389752
769 6.407718504640798
770 6.396069087200885
771 6.384453917758947
772 6.372872869222192
773 6.361325814821612
774 6.349812628403015
775 6.33833318388497
776 6.326887355731742
777 6.315475018654979
778 6.30409604785018
779 6.292750318687467
780 6.281437707177247
781 6.270158089471806
782 6.2589113421563525
783 6.2476

In [1]:
# np.save('eta', ETA_), np.save('x', X_)

#### Добавьте несколько оценок фильмов от себя. Файл movie_ids.txt содержит индексы каждого из фильмов.

In [16]:
with open(files_paths[1], 'rt', encoding='utf-8', errors='ignore') as f:
    films = [(int(film.split(' ')[0]), ' '.join(film.split(' ')[1:]))
             for film in f.read().split('\n')[:-1]]
    df_films = pd.DataFrame.from_records(films, columns=['film_index', 'film_name'])

In [17]:
i = 360
df_films.iloc[i:i+20]

,film_index,film_name
360,361,Incognito (1997)
361,362,Blues Brothers 2000 (1998)
362,363,Sudden Death (1995)
363,364,Ace Ventura: When Nature Calls (1995)
364,365,Powder (1995)
365,366,Dangerous Minds (1995)
366,367,Clueless (1995)
367,368,Bio-Dome (1996)
368,369,Black Sheep (1996)
369,370,Mary Reilly (1996)


#### Сделайте рекомендации для себя. Совпали ли они с реальностью?

In [ ]:
(1, 5), (27, 5), (50, 4), (96, 5) (373, 4)

In [44]:
def get_most_similar(film_index, X, n_best=5):
    film_vector = X[film_index - 1]
    similiarities = X.dot(film_vector) / (np.linalg.norm(X, axis=1) * np.linalg.norm(film_vector))
    best_indexes = np.argsort(similiarities)[-(n_best + 1):][::-1][1:]
    print('best matches for: ' + df_films.iloc[film_index - 1].values.astype(str)[1])
    print(df_films.iloc[best_indexes].values)

In [56]:
get_most_similar(50, X_)

best matches for: Star Wars (1977)
[[550 'Die Hard: With a Vengeance (1995)']
 [248 'Grosse Pointe Blank (1997)']
 [281 'River Wild, The (1994)']
 [778 'Don Juan DeMarco (1995)']
 [672 'Candyman (1992)']]


In [70]:
get_most_similar(1, X_)

best matches for: Toy Story (1995)
[[268 'Chasing Amy (1997)']
 [429 'Day the Earth Stood Still, The (1951)']
 [45 'Eat Drink Man Woman (1994)']
 [868 'Hearts and Minds (1996)']
 [622 'Swiss Family Robinson (1960)']]


In [18]:
my_marks = np.zeros(X_.shape[0])
my_marks[0] = 5
my_marks[26] = 5
my_marks[49] = 4
my_marks[95] = 5
my_marks[372] = 4

In [26]:
Y1, R1 = np.zeros((Y.shape[0], Y.shape[1]+1)), np.zeros((R.shape[0], R.shape[1]+1))
Y1[:,:Y.shape[1]] = Y
R1[:,:R.shape[1]] = R
Y1[:,:-1] = my_marks
R1[:,:-1] = (my_marks != 0).astype(int)

In [27]:
nU1, nM1 = Y1.shape

ETA1 = np.random.randn(nU1, n)
X1 = np.random.randn(nM1, n)

In [30]:
ETA1.shape

(1682, 10)

In [33]:
ETA1, X1 = fit(ETA1, X1, Y1, R1, eps=1e-5, lambda_=0.005, 
               lr=0.0005, maxiter=1000)

0 37114.06224215776
1 1005.543015752919
2 67.66501235090254
3 33.27664063760312
4 34.863323655954446
5 38.58215284193284
6 42.81334779705503
7 47.51303859791369
8 52.72676006432448
9 58.509704613883514
10 64.9228901957249
11 72.0335082164529
12 79.9154820550757
13 88.6500449354644
14 98.32633386774978
15 109.04199618507118
16 120.90380007098429
17 134.02823630212515
18 148.5420941026532
19 164.58298899323563
20 182.2998145777965
21 201.85308319458272
22 223.41511217021616
23 247.1700029971107
24 273.3133500986005
25 302.05160404281924
26 333.60100133917877
27 368.18595967396686
28 406.03682428289903
29 447.3868390255084
30 492.4682059930201
31 541.5070919371356
32 594.7174407940474
33 652.2934620514716
34 714.4006881325331
35 781.1655343462917
36 852.6633564944786
37 928.9050880041323
38 1009.8226537896553
39 1095.2535036156623
40 1184.924782449074
41 1278.4378540093749
42 1375.2541058415227
43 1474.6831724488875
44 1575.8748927818378
45 1677.816438102629
46 1779.3360693436553
47 1879.

344 0.00032473433871871293
345 0.000324732678421924
346 0.0003247310195888531
347 0.00032472936212002423
348 0.00032472770581648547
349 0.00032472605059297166
350 0.0003247243963215851
351 0.0003247227429170607
352 0.00032472109027992246
353 0.0003247194382822727
354 0.00032471778698095477
355 0.0003247161361628059
356 0.0003247144858704587
357 0.0003247128360044371
358 0.0003247111865647412
359 0.00032470953743768405
360 0.0003247078886658983
361 0.0003247062401783296
362 0.00032470459193234547
363 0.000324702943913735
364 0.00032470129613670906
365 0.0003246996485160025
366 0.0003246980010516154
367 0.0003246963537861802
368 0.0003246947066060102
369 0.00032469305958215955
370 0.0003246914126577849
371 0.0003246897658328862
372 0.0003246881190790418
373 0.00032468647243888427
374 0.000324684825869781
375 0.00032468317931488855
376 0.00032468153290210466
377 0.0003246798864893208
378 0.00032467824014759117
379 0.0003246765938484941
380 0.00032467494756360793
381 0.0003246733013781977


658 0.0003242176811824038
659 0.000324216037498104
660 0.00032421439381380424
661 0.00032421275012950446
662 0.0003242111064736264
663 0.00032420946280353746
664 0.0003242078191476594
665 0.0003242061754917813
666 0.00032420453189274667
667 0.00032420288822265775
668 0.00032420124460941224
669 0.0003241996010103776
670 0.0003241979573971321
671 0.00032419631379809744
672 0.00032419467021327364
673 0.0003241930266426607
674 0.0003241913830578369
675 0.0003241897395014348
676 0.0003241880959308219
677 0.0003241864524028415
678 0.00032418480886065026
679 0.0003241831653042482
680 0.0003241815218046895
681 0.00032417987829092
682 0.00032417823477715046
683 0.0003241765912775918
684 0.00032417494780645484
685 0.00032417330430689617
686 0.00032417166082154836
687 0.00032417001736462225
688 0.00032416837390769615
689 0.0003241667304649809
690 0.00032416508702226565
691 0.00032416344359376126
692 0.000324161800151046
693 0.00032416015673675247
694 0.00032415851335088064
695 0.00032415686992237

968 0.00032370852686369744
969 0.0003237068856805081
970 0.0003237052445399513
971 0.00032370360338518367
972 0.00032370196225883774
973 0.0003237003211040701
974 0.000323698679991935
975 0.00032369703887979995
976 0.000323695397753454
977 0.00032369375666974065
978 0.0003236921155860273
979 0.00032369047448810306
980 0.00032368883341860055
981 0.00032368719234909804
982 0.0003236855512938064
983 0.0003236839102385147
984 0.0003236822692116448
985 0.000323680628170564
986 0.00032367898714369403
987 0.0003236773461168241
988 0.000323675705104165
989 0.00032367406410571675
990 0.00032367242312147937
991 0.000323670782137242
992 0.0003236691411530046
993 0.00032366750019718893
994 0.00032366585921295155
995 0.0003236642182855576
996 0.0003236625773297419
997 0.0003236609363881371
998 0.0003236592954607431
999 0.00032365765456177087


In [50]:
my_recomends = ETA1.dot(X1.T)[:, -1]
sorted_my_recomends = np.argsort(my_recomends)[::-1]
df_films.iloc[sorted_my_recomends[:50]]

,film_index,film_name
1203,1204,To Be or Not to Be (1942)
318,319,Everyone Says I Love You (1996)
661,662,Somewhere in Time (1980)
785,786,"Perez Family, The (1995)"
1543,1544,It Takes Two (1995)
1110,1111,Double Happiness (1994)
956,957,Pushing Hands (1992)
1411,1412,Land Before Time III: The Time of the Great Gi...
86,87,Searching for Bobby Fischer (1993)
1108,1109,Death and the Maiden (1994)


#### Также обучите модель с помощью сингулярного разложения матриц. Отличаются ли полученные результаты?

In [40]:
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds

Y_sparce = csc_matrix(Y1, dtype=float)
u, s, v = svds(Y_sparce, k=10)

In [45]:
get_most_similar(50, v.T)

best matches for: Star Wars (1977)
[[96 'Terminator 2: Judgment Day (1991)']
 [27 'Bad Boys (1995)']
 [373 'Judge Dredd (1995)']
 [1 'Toy Story (1995)']
 [254 'Batman & Robin (1997)']]


In [46]:
get_most_similar(1, v.T)

best matches for: Toy Story (1995)
[[96 'Terminator 2: Judgment Day (1991)']
 [27 'Bad Boys (1995)']
 [50 'Star Wars (1977)']
 [373 'Judge Dredd (1995)']
 [326 'G.I. Jane (1997)']]


In [49]:
my_recomends1 = u.dot(v)[:, -1]
sorted_my_recomends1 = np.argsort(my_recomends1)[::-1]
df_films.iloc[sorted_my_recomends1[:50]]

,film_index,film_name
819,820,Space Jam (1996)
613,614,Giant (1956)
683,684,In the Line of Fire (1993)
1192,1193,Before the Rain (Pred dozhdot) (1994)
1475,1476,Raw Deal (1948)
429,430,Duck Soup (1933)
559,560,"Kid in King Arthur's Court, A (1995)"
1677,1678,Mat' i syn (1997)
1139,1140,"Road to Wellville, The (1994)"
674,675,"Nosferatu (Nosferatu, eine Symphonie des Graue..."
